In [1]:
import glob
import json
import os
import re
from pathlib import Path

## Создание аннотаций в формате JSON

Создание аннотаций без учета разделения на train, test

In [ ]:
data = {}
current_name = None
pattern = r".*\\.*"

with open("data/metadata/test_set.txt", "r", encoding="Windows 1251") as file:
    for line in file:
        line = line.strip()
        if re.match(pattern, line):
            current_name = line
            data[current_name] = []
        else:
            x, y, cat = line.split(", ")
            data[current_name].append((int(x), int(y), int(cat)))


with open("test.json", "w") as f:
    json.dump(data, f, indent=4)

data = {}
current_name = None
pattern = r".*\\.*"

with open("data/metadata/set.txt", "r", encoding="Windows 1251") as file:
    for line in file:
        line = line.strip()
        if re.match(pattern, line):
            current_name = line
            data[current_name] = []
        else:
            x, y, cat = line.split(", ")
            data[current_name].append((int(x), int(y), int(cat)))


with open("train.json", "w") as f:
    json.dump(data, f, indent=4)

Создание аннотаций c разделением на train, test

In [ ]:
with open("train.json", "r", encoding="utf-8") as f:
    train_ann = json.load(f)

dataset_dir = Path("./data/FRAMES/")

total_subfolders = len([x for x in dataset_dir.glob("*") if x.is_dir()])
train_new = {}
test_new = {}
train_keys = list(train_ann.keys())
k_train = 0.8
int(total_subfolders * k_train)
k = 0

for subfolder in sorted(list(dataset_dir.glob("*"))):
    if subfolder.is_dir():
        if k <= int(total_subfolders * k_train):
            for img_path in subfolder.rglob("*.bmp"):
                img_path = str(img_path).encode("UTF-8").decode("UTF-8")
                img_path = str(img_path).replace(dataset_dir.as_posix() + "/", "")
                img_path = img_path.replace(".bmp", ".frame")
                img_path = img_path.replace("/", "\\")
                if img_path in train_keys:
                    labels = train_ann[img_path]
                    train_new[img_path] = labels
        else:
            for img_path in subfolder.rglob("*.bmp"):
                img_path = str(img_path).replace(dataset_dir.as_posix() + "/", "")
                img_path = img_path.replace(".bmp", ".frame")
                img_path = img_path.replace("/", "\\")

                if img_path in train_keys:
                    labels = train_ann[img_path]
                    test_new[img_path] = labels
    k += 1

In [ ]:
with open("train.json", "w", encoding="utf8") as f:
    json.dump(train_new, f, indent=4, ensure_ascii=False)
with open("test.json", "w", encoding="utf8") as f:
    json.dump(test_new, f, indent=4, ensure_ascii=False)
print(len(train_new), len(test_new))

## Создание ключ-карты классов

In [ ]:
label_map = {
    0: "не дефект",
    1: "потертость",
    2: "черная точка",
    3: "плена",
    4: "маркер",
    5: "грязь",
    6: "накол",
    7: "н.д. накол",
    8: "микровыступ",
    9: "н.д. микровыступ",
    10: "вмятина",
    11: "мех.повреждение",
    12: "риска",
    13: "царапина с волчком",
}
with open("label_map.json", "w", encoding="utf8") as f:
    json.dump(label_map, f, indent=4)